In [2]:
# load latitude and longitude
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from scipy.cluster.vq import kmeans2, whiten
from sklearn.neighbors import NearestNeighbors 
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:

#from geopy.distance import great_circle
#from shapely.geometry import MultiPoint


X=df.loc[:,['latitude','longitude']]

x, y = kmeans2(X, 500, iter = 20)  #<--- I randomly picked 500 clusters
plt.scatter( X.loc[:,'longitude'], X.loc[:,'latitude'], c=y, alpha=0.33333);

# so we have centroids, there are 500 of them
# i will save them into csv file

df = pd.DataFrame(x)
df.to_csv("500_clusters.csv", encoding="utf-8",index=True, header = ['latitude', 'longitude'])

C:\Apps\Anaconda3\lib\site-packages\scipy\cluster\vq.py:653: UserWarning: One of the clusters is empty. Re-run kmean with a different initialization.
  warnings.warn("One of the clusters is empty. "


In [21]:
# I will assign for each image label from each cluster

df = pd.read_csv('dm_clean1.csv')
df_clusters = pd.read_csv("500_clusters.csv")
df = df.dropna()

X = df_clusters.as_matrix(columns=['latitude', 'longitude'])
nbrs = NearestNeighbors(n_neighbors=1,algorithm='ball_tree').fit(X)#<- KNN

X1 = df.as_matrix(columns=['latitude', 'longitude'])
distances , indices = nbrs.kneighbors(X1)

#indices = pd.DataFrame({'clusters':indices})
df['clusters'] = indices

df.to_csv("data_500clusters.csv", encoding="utf-8",index=False)

In [22]:
df = pd.read_csv("data_500clusters.csv")

In [23]:
# concatenate all the texts to the appropriate cluster

df = df.drop(['latitude','longitude','photoID', 'userID', 'photoLink', 'DateTaken'], axis=1)

In [24]:
# function to be applied that takes a row and a list of columns 
# to concatenate
def concat_text(row, cols):
    # The real work is done here
    return " ".join([" ".join([str(x) for x in y if x]) for y in row[cols].values])

result = df.groupby('clusters').apply(concat_text, ['photoTags']) # groupby and apply

In [25]:
result.to_csv("500clusters_with_tags.csv", encoding="utf-8", header=['photoTags'])

In [28]:
# I will save all the tags for 500 clusters into 500 files
# dataset = load_files(container_path = 'data/txt-combined/',shuffle=False)

df = pd.read_csv("500clusters_with_tags.csv")

print(df)

     clusters                                          photoTags
0           0  idaho backpacking sawtoothmountains sawtoothwi...
1           1  anais park ramenskoe park ramenskoe road  coun...
2           4  building architecture vintage 2006 railwaystat...
3           5           thenationalarchivesuk africathroughalens
4           6  family india me kolkata calcutta middleschool ...
5           7  sunset sun beach washington sand graysharbor o...
6           8  city montana downtown mt miles foe eaglesclub ...
7          10  blackandwhite bw white building church archite...
8          12  voyage travel church  iceland glise canonef247...
9          13                             pakistan karachi sindh
10         15  autumn canada sign saskatoon saskatchewan neig...
11         16  khajuraho khajuraho madhyapradesh people peopl...
12         18  starlake goldriver vancouverislandfallnaturepa...
13         19  egypt egitto egypt egitto egypt egitto abusimb...
14         20  muffin hot

In [3]:
# so we have 500 clusters with correspondanse values of latitude and longitude , and concatenated texts,
# we compare this clusters with the texts of the test sets 
train_data = pd.read_csv('500clusters_with_tags.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer()
train_c_matrix = _vect.fit_transform(train_data_)
vocab = _vect.vocabulary_

nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
nbrs.fit(train_c_matrix)

test_data = pd.read_csv('test2_all.csv')
# delete all data with empty tags from test data
#test_data.dropna(subset=['photoTags'], inplace=True)
test_data['photoTags'].replace( np.nan,'', inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [4]:
df = pd.read_csv('500_clusters.csv')
import datetime
predicted_lt = []
predicted_lng = []
#indexes = []
print(datetime.datetime.now())

for i in range(0,len(test_data_)):
    #distances, indices = nbrs.kneighbors(svd_test_matrix[[i]])
    
    distances, indices = nbrs.kneighbors(test_c_matrix[i])
    #distances, indices = tree.query([svd_test_matrix[i]], k=1)
    ind = indices[0][0]
    #indexes.append(indices[0][0])
    if i%100==0:
        print(i)
        print(datetime.datetime.now())            
    #print('ind',ind)
    #print(distances)
    
    cl = train_data.loc[ind,'clusters']
    lt = df.loc[cl,'latitude']
    predicted_lt.append(lt)
    lng = df.loc[cl,'longitude']
    predicted_lng.append(lng)
    

    #df_lt = pd.DataFrame([lt], columns=['lt'])
    #df_ln = pd.DataFrame([lng], columns=['ln'])
    
    #data_lt = data_lt.append(df_lt)
    #data_ln = data_ln.append(df_ln)
    #print('test:', test_data.loc[i,'photoTags'])
    #print('train:', train_data.loc[ind,'photoTags'])
    #print('-----------------------------')
    

test_data['lt_predicted'] = predicted_lt
test_data['lng_predicted'] = predicted_lng

test_data.to_csv("predicted_2_clusters.csv", encoding="utf-8")

data_predicted = pd.read_csv('predicted_2_clusters.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

data_predicted.to_csv("predicted_2_clusters.csv", encoding="utf-8")

2018-05-16 08:38:47.562685
0
2018-05-16 08:38:47.912677
100
2018-05-16 08:38:58.413041
200
2018-05-16 08:39:08.933384
300
2018-05-16 08:39:19.483710
400
2018-05-16 08:39:29.994054
500
2018-05-16 08:39:40.474396
600
2018-05-16 08:39:50.934760
700
2018-05-16 08:40:01.415103
800
2018-05-16 08:40:11.816595
900
2018-05-16 08:40:22.196963
1000
2018-05-16 08:40:32.767373
1100
2018-05-16 08:40:43.297762
1200
2018-05-16 08:40:53.788190
1300
2018-05-16 08:41:04.398600
1400
2018-05-16 08:41:14.999018
1500
2018-05-16 08:41:25.709426
1600
2018-05-16 08:41:36.964784
1700
2018-05-16 08:41:47.585196
1800
2018-05-16 08:41:58.125625
1900
2018-05-16 08:42:08.626030
2000
2018-05-16 08:42:19.046418
2100
2018-05-16 08:42:29.436822
2200
2018-05-16 08:42:39.837247
2300
2018-05-16 08:42:50.247651
2400
2018-05-16 08:43:00.648035
2500
2018-05-16 08:43:11.088461
2600
2018-05-16 08:43:21.538847
2700
2018-05-16 08:43:31.993924
2800
2018-05-16 08:43:42.454329
2900
2018-05-16 08:43:52.884754
3000
2018-05-16 08:44:03.

In [5]:
data_predicted = pd.read_csv('predicted_2_clusters.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

data_predicted.to_csv("predicted_2_clusters_errors.csv", encoding="utf-8")
radius = [0.00001, 0.0001, 0.01, 0.1, 1, 10, 100, 400]

# 1 latitude = 111 km , so we will round it to 100 km
answers = []
for el in radius: 
    answers.append(data_predicted['error'][data_predicted['error']<el].count())
accuracy = []
amount = len(data_predicted)
print(radius)
print(answers)
for i in range(0,8):
    accuracy.append(answers[i]/amount)
print(accuracy)

[1e-05, 0.0001, 0.01, 0.1, 1, 10, 100, 400]
[0, 0, 5, 433, 5131, 10205, 16107, 21200]
[0.0, 0.0, 0.00023584905660377359, 0.020424528301886794, 0.24202830188679245, 0.4813679245283019, 0.75976415094339622, 1.0]


In [ ]:
cl